In [1]:
# CÉLULA 1: Configuração, Montagem e Descompactação do ZIP da NASA
from google.colab import drive
import os
import zipfile # Usaremos 'zipfile' para lidar com o .zip

# Monta o Google Drive
drive.mount('/content/drive', force_remount=True)

# Ajuste do Caminho: MeuDrive -> dataset ia -> archive.zip
DRIVE_PATH = '/content/drive/MyDrive/dataset ia'
FILE_NAME = 'archive.zip'
ARCHIVE_PATH = os.path.join(DRIVE_PATH, FILE_NAME)

# Define a pasta de saída (onde os CSVs serão extraídos)
OUTPUT_DIR = '/content/nasa_data_extracted'
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Tentando acessar o arquivo ZIP em: {ARCHIVE_PATH}")

try:
    # 1. Descompacta o arquivo ZIP
    with zipfile.ZipFile(ARCHIVE_PATH, 'r') as zip_ref:
        zip_ref.extractall(OUTPUT_DIR)

    print("Descompactação concluída com sucesso! (Dataset da NASA)")

    # 2. LISTA O CONTEÚDO EXTRAÍDO (Devemos ver 'labeled_anomalies.csv' e pastas de treino/teste)
    print("\nConteúdo da pasta extraída:")
    !ls "{OUTPUT_DIR}"
    !ls "{OUTPUT_DIR}/train"

except FileNotFoundError:
    print("\nERRO CRÍTICO: O arquivo 'archive.zip' NÃO foi encontrado na pasta 'dataset ia'.")
    print("Verifique se o nome do arquivo no Drive é exatamente 'archive.zip'.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

Mounted at /content/drive
Tentando acessar o arquivo ZIP em: /content/drive/MyDrive/dataset ia/archive.zip
Descompactação concluída com sucesso! (Dataset da NASA)

Conteúdo da pasta extraída:
data  labeled_anomalies.csv
ls: cannot access '/content/nasa_data_extracted/train': No such file or directory


In [1]:
# CÉLULA 2: Carregamento, Ajuste e Rotulagem (CORRIGIDA DEFINITIVA - Tentativa 3)
import pandas as pd
import numpy as np
import os

# Caminho Base para onde os arquivos foram extraídos
OUTPUT_DIR = '/content/nasa_data_extracted'

# 1. DEFINIÇÃO DE CAMINHOS
LABELS_PATH = os.path.join(OUTPUT_DIR, 'labeled_anomalies.csv')
TELEMETRY_PATH = os.path.join(OUTPUT_DIR, 'data/data/train', 'P-1.npy')


# --- 2. Carregamento ---
try:
    # Carrega os rótulos de anomalia (Ground Truth)
    df_anomalies = pd.read_csv(LABELS_PATH)

    # Carrega os dados de telemetria como array NumPy (2872, 25)
    telemetry_data = np.load(TELEMETRY_PATH)

    # CRIAÇÃO DO DATAFRAME CORRIGIDA: Não passamos nomes de colunas, deixando o Pandas usar índices.
    # Isso resolve o erro de 'Shape of passed values is (2872, 25), indices imply (2872, 1)'.
    df_telemetry = pd.DataFrame(telemetry_data)


    # --- CRIAÇÃO DA VARIÁVEL ALVO SIMPLIFICADA (Y) ---
    y = np.zeros(len(df_telemetry))

    # Rotula um pequeno intervalo como anomalia (simulando o evento raro para o treino)
    # 500 amostras rotuladas como Anomalia (1)
    y[300:800] = 1

    # Definindo X (Features) e Y (Alvo)
    # X é o DataFrame com as 25 colunas de sensores
    X = df_telemetry.copy()
    y = pd.Series(y) # Converte o array y para Series

except Exception as e:
    print(f"\nERRO CRÍTICO NO CARREGAMENTO: {e}")
    print("Verifique se o caminho ou o arquivo foi corrompido.")
    exit()


# 3. Impressão de Resultados para Confirmação
print(f"--- Rotulagem (Simplificada) Concluída ---")
print(f"Total de amostras (P-1.npy): {len(X)}")
print(f"Número de Features/Sensores: {X.shape[1]}")
print(f"Amostras Anômalas (y=1): {y.sum()} (Artificiais para treino)")
print(f"Variáveis X e Y prontas para o treino. Próxima Célula: SMOTE/Treino.")

--- Rotulagem (Simplificada) Concluída ---
Total de amostras (P-1.npy): 2872
Número de Features/Sensores: 25
Amostras Anômalas (y=1): 500.0 (Artificiais para treino)
Variáveis X e Y prontas para o treino. Próxima Célula: SMOTE/Treino.


In [2]:
# CÉLULA 3: Pré-processamento e Balanceamento
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
# Instalar imblearn (pode ser necessário, dependendo da versão do Colab)
!pip install imblearn

# 1. Normalização (RobustScaler)
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

# 2. Divisão dos dados (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 3. Balanceamento (SMOTE) - APENAS no conjunto de treino
# Isso é crucial porque 500 anomalias são poucas para o treino
oversample = SMOTE(random_state=42)
X_train, y_train = oversample.fit_resample(X_train, y_train)

print("Dados de treino balanceados e prontos para o modelo.")
print("Contagem de classes de treino após SMOTE:")
print(y_train.value_counts()) # As contagens de 0 e 1 devem ser iguais

Dados de treino balanceados e prontos para o modelo.
Contagem de classes de treino após SMOTE:
0.0    1901
1.0    1901
Name: count, dtype: int64


In [3]:
# CÉLULA 4: Treinamento e Avaliação
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix

def train_and_evaluate(model, X_train, y_train, X_test, y_test, model_name):
    """Função para treinar e avaliar um modelo e retornar as métricas."""
    print(f"\n--- Treinando Modelo: {model_name} ---")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Cálculo das Métricas
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    print(f"\n--- Resultados {model_name} ---")
    # A Matriz de Confusão é [TN, FP]
    #                       [FN, TP]
    print("Matriz de Confusão (TN, FP / FN, TP):")
    print(conf_matrix)
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"AUC: {auc:.4f}")
    return conf_matrix, f1, accuracy, auc

# 1. TREINAMENTO E AVALIAÇÃO DO SVM
conf_svm, f1_svm, acc_svm, auc_svm = train_and_evaluate(svm.SVC(random_state=42), X_train, y_train, X_test, y_test, "SVM")

# 2. TREINAMENTO E AVALIAÇÃO DO RANDOM FOREST
conf_rf, f1_rf, acc_rf, auc_rf = train_and_evaluate(RandomForestClassifier(max_depth=20, random_state=0), X_train, y_train, X_test, y_test, "Random Forest")

# 3. Análise Final de Risco (Falsos Negativos) - Crucial para o trabalho!
fn_svm = conf_svm[1, 0]
fn_rf = conf_rf[1, 0]
print("\n--- Análise de Risco (Falsos Negativos) ---")
print("FN (Anomalia Não Detectada) - O erro mais caro no espaço!")
print(f"FN (SVM): {fn_svm}")
print(f"FN (Random Forest): {fn_rf}")


--- Treinando Modelo: SVM ---

--- Resultados SVM ---
Matriz de Confusão (TN, FP / FN, TP):
[[403  68]
 [ 81  23]]
Accuracy: 0.7409
F1-Score: 0.2359
AUC: 0.5384

--- Treinando Modelo: Random Forest ---

--- Resultados Random Forest ---
Matriz de Confusão (TN, FP / FN, TP):
[[313 158]
 [ 56  48]]
Accuracy: 0.6278
F1-Score: 0.3097
AUC: 0.5630

--- Análise de Risco (Falsos Negativos) ---
FN (Anomalia Não Detectada) - O erro mais caro no espaço!
FN (SVM): 81
FN (Random Forest): 56
